In [1]:
import os
import pandas as pd

把工具预测的结果进行统一处理

In [3]:
import numpy as np

tools_prediction_data_directory = '/mnt/zt/Dopaap/calculate_webtools_score/HLA-I_dataset/web_tools_prediction_group_by_allele'
processed_for_calculate_dir = '/mnt/zt/Dopaap/calculate_webtools_score/HLA-I_dataset/processed_for_calculate'

tools_prediction_columns_modify_dict = {
    'SMM': 'smm',
    'SMMPMBEC': 'smmpmbec',
    'ANN4': 'ann',
    'NetMHCpan': 'netmhcpan_ba'
}

tools_prediction_columns = list(tools_prediction_columns_modify_dict.keys())

tools_prediction_need_columns = ['allele','peptide', 'QM'] + tools_prediction_columns

for file in os.listdir(tools_prediction_data_directory):
    allele = file.split('.')[0]
    df_tools_prediction = pd.read_csv(os.path.join(tools_prediction_data_directory, file))
    
    for col in tools_prediction_need_columns:
        if col not in df_tools_prediction.columns:
            df_tools_prediction[col] = pd.NA
            
    df_tools_prediction = df_tools_prediction[tools_prediction_need_columns]
    df_tools_prediction.rename(columns=tools_prediction_columns_modify_dict, inplace=True)
    
    for tools_prediction in list(tools_prediction_columns_modify_dict.values()):
        if not df_tools_prediction[tools_prediction].isna().any():
            df_tools_prediction['Normalized_QM'] = np.maximum(1 - (np.log(df_tools_prediction['QM']) / np.log(50000)), 0.0)
            df_tools_prediction[f'{tools_prediction}_Normalized'] = 1 - (np.log(df_tools_prediction[tools_prediction]) / np.log(50000))
    
    # 保存合并后的结果到 CSV 文件
    df_tools_prediction.to_csv(f'{processed_for_calculate_dir}/{allele}.csv', index=False)
    

把Dopaap预测结果合并到其它工具的预测结果

In [4]:
import pandas as pd
import os

processed_tools_prediction_data_directory = '/mnt/zt/Dopaap/calculate_webtools_score/HLA-I_dataset/processed_for_calculate'
dopaap_retrain_pred_dict = '/mnt/zt/Dopaap/calculate_webtools_score/HLA-I_dataset/Dopaap_result/group_by_allele'

for allele in os.listdir(dopaap_retrain_pred_dict):
    df_tools_pred = pd.read_csv(os.path.join(processed_tools_prediction_data_directory, f'{allele}.csv'))
    df_dopaap_retrain = pd.read_csv(f'{dopaap_retrain_pred_dict}/{allele}/predict_result.csv')[['Pred_Norm_QM']].rename(columns={'Pred_Norm_QM': 'Dopaap-new-super-params_IC50_Normalized'})
    df_tools_pred['Dopaap-new-super-params_IC50_Normalized'] = df_dopaap_retrain
    df_tools_pred.to_csv(os.path.join(processed_tools_prediction_data_directory, f'{allele}.csv'), index=False)